For this task,  we will be using [SAMSum](https://huggingface.co/datasets/samsum) dataset. The SAMSum dataset contains about 16k messenger-like conversations with summaries.

The dataset contains following data fields:
* `dialogue`: text of dialogue.
* `summary`: human written summary of the dialogue.
* id: unique id of an example.


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [5]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.0 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

dataset = load_dataset("samsum")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [7]:
for i in range(10):
    print(len(dataset['train']['dialogue'][i]), len(dataset['train']['summary'][i]))
    print(dataset['train'][i])

94 56
{'id': '13818513', 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)", 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}
111 61
{'id': '13728867', 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great', 'summary': 'Olivia and Olivier are voting for liberals in this election. '}
528 77
{'id': '13681000', 'dialogue': "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like

### Task 1. Dataset processing [1 point]

**a. Vocabulary**

You need to create a vocabulary:
*   Create a vocabulary by using tokens in `dialogue` and `summary` fields of the training data.
*   Add `<pad>`, `<unk>` , `<sos>`, and `<eos> `special tokens.
*   Make index for `<unk>` token default for the vocabulary.

In [8]:
len(dataset['train'])

14732

In [9]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# tokenize using tokenizer
for i in range(len(dataset['train'])):
    train_dialogue_tokens = word_tokenize(dataset['train'][i]['dialogue'].lower())
    train_summary_tokens = word_tokenize(dataset['train'][i]['summary'].lower())
unique_train_dialogue_tokens = set(train_dialogue_tokens)
unique_train_summary_tokens = set(train_summary_tokens)

# tokenize using split
for i in range(len(dataset['train'])):
    train_dialogue_splits = (dataset['train'][i]['dialogue'].lower().split())
    train_summary_splits = (dataset['train'][i]['summary'].lower().split())
train_dialogue_splits_unique= set(train_dialogue_splits)
train_summary_splits_unique= set(train_summary_splits)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
print("Tokenizer based no of dialogue tokens:", len(train_dialogue_tokens))
print("Tokenizer unique no of dialogue tokens:", len(unique_train_dialogue_tokens))
print("Tokenizer based no of summary tokens:", len(train_summary_tokens))
print("Tokenizer unique no of summary tokens:", len(unique_train_summary_tokens))
print()
print("Split based no of dialogue tokens:",len(train_dialogue_splits))
print("Split unique no of dialogue tokens:",len(train_dialogue_splits_unique))
print("Split based no of summary tokens:",len(train_summary_splits))
print("Split unique no of summary tokens:",len(train_summary_splits_unique))

Tokenizer based no of dialogue tokens: 307
Tokenizer unique no of dialogue tokens: 136
Tokenizer based no of summary tokens: 43
Tokenizer unique no of summary tokens: 31

Split based no of dialogue tokens: 240
Split unique no of dialogue tokens: 133
Split based no of summary tokens: 37
Split unique no of summary tokens: 31


In [11]:
special_tokens = ['<unk>', '<pad>', '<sos>', '<eos>']
train_tokens = train_dialogue_tokens + train_summary_tokens
unique_train_tokens = set(train_tokens)
print("Unique tokenizer tokens", len(unique_train_tokens))

# Initialize the vocabulary dictionary with special tokens first
train_vocab = {token: index for index, token in enumerate(special_tokens)}
# Update the vocabulary with other tokens, ensuring special tokens stay at the beginning
for token in unique_train_tokens:
    if token not in train_vocab:
        train_vocab[token] = len(train_vocab)
print("Unique all tokens:", len(train_vocab))

Unique tokenizer tokens 147
Unique all tokens: 151


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

**b. Process the datasets**

For each dataset split:
*   Convert the dataset into token IDs by using created vocabulary
*   Calculate source (`dialogue`) and target(`summary`) lengths of each sample

In [13]:
# tokenize the entire dataset ie all splits
from nltk.tokenize import word_tokenize

splits = ['train', 'validation', 'test']
tokenized_data = {split: {'dialogue': [], 'summary': []} for split in splits}

for split in splits:
    for i in range(len(dataset[split])):
        # Tokenize and store dialogue tokens
        dialogue_tokens = word_tokenize(dataset[split][i]['dialogue'].lower())
        tokenized_data[split]['dialogue'].append(dialogue_tokens)

        # Tokenize and store summary tokens
        summary_tokens = word_tokenize(dataset[split][i]['summary'].lower())
        tokenized_data[split]['summary'].append(summary_tokens)


In [14]:
# Function to convert tokens to IDs
def tokens_to_ids(tokens, vocab):
    return [vocab.get(token, 0) for token in tokens]

In [15]:
# Initialize dictionaries to store the maximum lengths
max_lengths = {
    'dialogue': {split: 0 for split in splits},
    'summary': {split: 0 for split in splits},
}

# Convert to token IDs and calculate lengths
for split in splits:
    for i in range(len(tokenized_data[split]['dialogue'])):
        # Convert dialogue tokens to IDs and store
        dialogue_ids = tokens_to_ids(tokenized_data[split]['dialogue'][i], train_vocab)
        tokenized_data[split]['dialogue'][i] = dialogue_ids

        # Update the maximum dialogue length
        max_lengths['dialogue'][split] = max(max_lengths['dialogue'][split], len(dialogue_ids))

        # Convert summary tokens to IDs and store
        summary_ids = tokens_to_ids(tokenized_data[split]['summary'][i], train_vocab)
        tokenized_data[split]['summary'][i] = summary_ids

        # Update the maximum summary length
        max_lengths['summary'][split] = max(max_lengths['summary'][split], len(summary_ids))

In [35]:
# Prepare the dataset list in the expected format
def prepare_data_list(tokenized_data_split):
    return [{'dialogue': dialogue_ids, 'summary': summary_ids}
            for dialogue_ids, summary_ids in zip(tokenized_data_split['dialogue'], tokenized_data_split['summary'])]

train_data_list = prepare_data_list(tokenized_data['train'])
val_data_list = prepare_data_list(tokenized_data['validation'])
test_data_list = prepare_data_list(tokenized_data['test'])

# Now, use these lists to create your datasets
train_dataset = DialogueSummaryDataset(train_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)
val_dataset = DialogueSummaryDataset(val_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)
test_dataset = DialogueSummaryDataset(test_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)


In [17]:
max_lengths

{'dialogue': {'train': 995, 'validation': 663, 'test': 644},
 'summary': {'train': 73, 'validation': 68, 'test': 71}}

In [20]:
print(type(tokenized_data['train']))
print(type(train_data_list[1]))
print(train_data_list[1])
# If it's a dictionary or another type, you may need to convert or extract the data differently.


<class 'dict'>
<class 'list'>
[[0, 0, 0, 68, 69, 0, 0, 0, 0, 112], [0, 68, 0, 75, 0, 98, 0, 50, 116, 0, 112], [0, 0, 0, 58, 0, 0, 0, 94, 0, 54, 129, 0, 0, 62, 112], [0, 0, 0, 0, 50, 0, 78, 0, 112, 0, 0, 0, 54, 0, 0, 0, 112, 0, 0, 0, 112], [0, 0, 0, 88, 0, 0, 0, 0, 0, 113, 0, 0, 123, 0, 112, 0, 0, 0, 0, 0, 54, 0, 112, 0, 0, 0, 35, 49, 54, 138, 112], [0, 0, 0, 0, 0, 0, 0, 64, 58, 0, 142, 0, 112, 0, 108, 0, 0, 0, 68, 99, 0, 76, 0, 0, 0, 113, 0, 0, 112], [0, 20, 18, 149, 0, 98, 0, 0, 54, 0, 0, 0, 78, 0, 0, 112, 0, 88, 0, 0, 0, 0, 0, 0, 54, 138, 88, 68, 0, 0, 54, 0, 112, 48, 75, 0, 54, 0, 0, 98, 123, 0, 0, 116, 0, 0, 0, 112], [0, 0, 0, 38, 0, 0, 0, 0, 86, 112, 0, 0, 58, 0, 112, 58, 0, 0, 58, 0, 54, 58, 0, 48, 0, 0, 50, 68, 0, 54, 0, 58, 0, 0, 112], [0, 0, 54, 0, 88, 0, 0, 0, 0, 88, 0, 0, 0, 0, 123, 0, 112], [0, 0, 0, 98, 123, 0, 54, 129, 54, 7, 0, 0, 120, 112, 48, 0, 97, 54, 58, 0, 0, 50, 0, 64, 0, 19, 0, 0, 88, 68, 0, 0, 0, 0, 0, 64, 58, 0, 54, 58, 0, 112], [0, 0, 0, 112, 0, 69, 0, 109, 78


**c. Automatic batching**

*   Calculate maximum length of the sources (SOURCE_LEN) and targets (TARGET_LEN)
*   Pad each source and target to their maximum length (SOURCE_LEN and TARGET_LEN, respectively) and return appropriate fields with collate function
*   Decide on batch size (e.g., 8, 16, 32, or any 2^n) and initialize DataLoader for each dataset split.

In [36]:
import torch
from torch.utils.data import DataLoader, Dataset

class DialogueSummaryDataset(Dataset):
    def __init__(self, data, vocab, source_max_len, target_max_len):
        self.data = data
        self.vocab = vocab
        self.source_max_len = source_max_len
        self.target_max_len = target_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        print(sample)
        assert isinstance(sample, dict), "Each sample should be a dictionary."

        # Get dialogue and summary token IDs from the sample
        dialogue_ids = sample['dialogue']
        summary_ids = sample['summary']

        # Pad the dialogue and summary to the maximum length
        dialogue_ids += [self.vocab['<pad>']] * (self.source_max_len - len(dialogue_ids))
        summary_ids += [self.vocab['<pad>']] * (self.target_max_len - len(summary_ids))

        return torch.tensor(dialogue_ids, dtype=torch.long), torch.tensor(summary_ids, dtype=torch.long)

# Calculate maximum lengths based on the preprocessed data
SOURCE_LEN = max_lengths['dialogue']['train']
TARGET_LEN = max_lengths['summary']['train']

# Initialize the dataset for the training split
#train_dataset = DialogueSummaryDataset(tokenized_data['train'], train_vocab, SOURCE_LEN, TARGET_LEN)
#train_dataset = DialogueSummaryDataset(train_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)

batch_size=batch_size = 8

# Initialize DataLoader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [37]:
# Initialize the dataset for the training split
#val_dataset = DialogueSummaryDataset(tokenized_data['validation'], train_vocab, SOURCE_LEN, TARGET_LEN)
#val_dataset = DialogueSummaryDataset(train_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)
batch_size=batch_size = 8

# Initialize DataLoader for the training dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [38]:
# Initialize the dataset for the training split
#test_dataset = DialogueSummaryDataset(test_data_list, train_vocab, SOURCE_LEN, TARGET_LEN)

batch_size=batch_size = 8

# Initialize DataLoader for the training dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [39]:
SOURCE_LEN, TARGET_LEN

(995, 73)

### Task 2. Encoder [1 point]

In the encoder, we will use a single-layer bidirectional LSTM. Additionally, we will define two linear layers to reduce the dimensionality of the concatenated hidden (`h_n`) and cell states (`c_n`). These layers will take the concatenation of the forward and backward hidden and cell states from the bidirectional LSTM as input and produce hidden states of dimensionality `hidden_size`.
  
**Forward Method:**
1. Pass the input sequences (`input`) through the bidirectional LSTM layer.
2. Concatenate the forward and backward hidden states along the last dimension.
3. Apply linear transformations to reduce the dimensionality of the concatenated hidden and cell states using the defined linear layers
4. Apply the ReLU activation function to the outputs of the linear layers.
5. Return the output sequence of the LSTM along with the final hidden and cell states.

**Additional Notes:**
- Ensure that the LSTM layer is configured to handle batched input sequences correctly
- Use PyTorch functions such as `torch.cat` and `unsqueeze` for tensor manipulations

In [ ]:
class Encoder(nn.Module):

    def __init__(self, input_size, hidden_size):
        super().__init__()

        #TODO: define a single-layer bidirectional LSTM
        self.lstm = ...

        #TODO: Define two linear layers with bias to reduce the dimensionality
             # of the concatenated hidden (h_n) and cell states (c_n) to `hidden_size`
        self.reduce_h_n = ...
        self.reduce_c_n = ...

    def forward(self, input, input_len):
        #TODO: pass the input to LSTM layer. If you want, you can use
             # pack_padded_sequence and pad_packed_sequence for input and output
        output, (h_n, c_n) = ...

        #TODO: Concatenate the forward and backward hidden and cell states
        h_n = ...
        c_n = ...

        #TODO:Apply linear transformations to reduce the dimensionality
        h_n = ...
        c_n = ...

        #TODO:Apply the ReLU activation function to the outputs of the linear layers
        h_n = ...
        c_n = ...

        return output, (h_n, c_n)

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        # Define a single-layer bidirectional LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)

        # Define linear layers to reduce the dimensionality of the concatenated hidden (h_n) and cell states (c_n)
        # The input features to the linear layers will be 2 * hidden_size because the LSTM is bidirectional
        self.reduce_h_n = nn.Linear(2 * hidden_size, hidden_size)
        self.reduce_c_n = nn.Linear(2 * hidden_size, hidden_size)

    def forward(self, input, input_len):
        # If using pack_padded_sequence, make sure to pack the input
        packed_input = nn.utils.rnn.pack_padded_sequence(input, input_len, batch_first=True, enforce_sorted=False)

        # Pass the packed input to the LSTM layer
        packed_output, (h_n, c_n) = self.lstm(packed_input)

        # Unpack the output if you have packed it earlier
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        # Concatenate the forward and backward hidden and cell states
        # h_n and c_n are of shape [num_layers * num_directions, batch, hidden_size]
        h_n = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
        c_n = torch.cat((c_n[-2, :, :], c_n[-1, :, :]), dim=1)

        # Apply linear transformations to reduce the dimensionality
        h_n = self.reduce_h_n(h_n)
        c_n = self.reduce_c_n(c_n)

        # Apply the ReLU activation function
        h_n = F.relu(h_n)
        c_n = F.relu(c_n)

        return output, (h_n, c_n)


### Task 3. Attention [2 points]

In the Attention model class, we will define linear projections (`Ea` and `Da`) to transform the encoder outputs and decoder input, respectively, to a common feature space. We will also define a linear layer (`Va`) to learn the importance of each encoder output when computing attention scores.

**Forward Method:**
  - Compute the attention scores using the Bahdanau attention mechanism (Equations 1 in the [paper](https://arxiv.org/pdf/1704.04368.pdf#page=2)). This involves:
    - Projecting the encoder outputs (`encoder_outputs`) and decoder input (`input`) to a common feature space.
    - Calculating the attention scores by adding the projected encoder outputs and decoder input and applying the tanh activation function followed by the linear layer (`Va`).
  - Compute the attention distribution by applying the softmax function to the masked attention scores (Equations 2 in the [paper](https://arxiv.org/pdf/1704.04368.pdf#page=2))
  - Compute the context vector by performing a weighted sum of the encoder outputs using the attention distribution (Equations 3 in the [paper](https://arxiv.org/pdf/1704.04368.pdf#page=3))
  - Return the attention distribution and the context vector.

**Additional Notes:**
- Ensure compatibility of dimensions throughout the operations, especially when performing tensor manipulations such as squeezing, unsqueezing, and summing.
- Use PyTorch functions such as `torch.bmm`, `torch.tanh`, `torch.softmax` operations for efficient implementation.

In [ ]:
class Attention(nn.Module):

    def __init__(self, hidden_size):
        super().__init__()

        #TODO: define linear layer WITHOUT bias to transform encoder output
        self.Ea = ...

        #TODO: define linear layer WITH bias to transform decoder input
        self.Da = ...

        #TODO: define linear layer WITHOUT bias to learn attention weights
        self.Va = ...


    def forward(self, input, encoder_outputs, pad_mask):

        #TODO: apply linear layer to transform encoder outputs
        enc = ...

        #TODO: apply linear layer to transform decoder input
        dec = ...

        #TODO: add transformed encoder and decoder features together to get attention scores
        scores = ...

        #TODO: apply the tanh activation function to attention scores
        scores = ...

        #TODO: apply the linear layer to attention scores
        scores = ....

        if pad_mask is not None:
            scores = scores.float().masked_fill_(
                pad_mask,
                float('-inf')
            ).type_as(scores)

        #TODO:Compute the attention weights by applying the softmax
        attn_dist = ...

        #Perform a batch matrix-matrix product of matrices stored in attn_dist and encoder_outputs
        context = ...

        #TODO: calculate weighted sum of context vector
        context = ...

        return attn_dist, context


In [70]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()

        # Define linear layer WITHOUT bias to transform encoder output
#        self.Ea = nn.Linear(hidden_size, hidden_size, bias=False)
        self.Ea = nn.Linear(1024, hidden_size, bias=False)


        # Define linear layer WITH bias to transform decoder input
        self.Da = nn.Linear(hidden_size, hidden_size, bias=True)

        # Define linear layer WITHOUT bias to learn attention weights
        self.Va = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, input, encoder_outputs, pad_mask):
        # Apply linear layer to transform encoder outputs
        print("Encoder outputs shape:", encoder_outputs.shape)
        enc = self.Ea(encoder_outputs)  # [batch_size, seq_len, hidden_size]

        # Apply linear layer to transform decoder input
        dec = self.Da(input)  # [batch_size, 1, hidden_size]

        # Add transformed encoder and decoder features together to get attention scores
        scores = torch.tanh(enc + dec.unsqueeze(1))  # [batch_size, seq_len, hidden_size]

        # Apply the linear layer to attention scores
        scores = self.Va(scores).squeeze(2)  # [batch_size, seq_len]

        # Apply mask
        if pad_mask is not None:
            pad_mask = pad_mask.unsqueeze(1)  # Adjusting the mask
            scores = scores.masked_fill(pad_mask, float('-inf'))

        # Compute the attention weights by applying the softmax
        attn_dist = torch.softmax(scores, dim=-1)  # [batch_size, seq_len]

        # Perform a batch matrix-matrix product of matrices stored in attn_dist and encoder_outputs
        context = torch.bmm(attn_dist.unsqueeze(1), encoder_outputs)  # [batch_size, 1, hidden_size]

        # Squeeze the context vector to remove the middle dimension
        context = context.squeeze(1)  # [batch_size, hidden_size]

        return attn_dist, context


### Task 4. Decoder with Attention [1 point]

The decoder will generate output sequences based on the attention mechanism, which allows it to focus on relevant parts of the input sequence.
We will define a single-layer unidirectional LSTM and use our `Attention` module to compute attention scores and context vectors. Additionally, we will define two linear layers to produce the vocabulary distribution.

**Forward Method:**
  - Pass the decoder input (`input`) through the LSTM decoder to get the output sequence, hidden and cell states (`h_n` and `c_n`).
  - Compute the attention distribution and context vector using the `Attention` module, passing the decoder hidden state, encoder outputs, and padding mask.
  - Produce vocabulary distribution (Equation 4 in the [paper](https://arxiv.org/pdf/1704.04368.pdf#page=3)):
    - Concatenate the decoder hidden state and context vector
    - Pass the concatenated vector through the linear layers `linear_v` and `linear_v_out` to obtain the vocabulary distribution over the output tokens.
    - Apply the softmax function to obtain the final probability distribution over the vocabulary.
  - Return the vocabulary distribution, attention distribution, context vector, and updated hidden and cell states.

In [ ]:
class AttentionDecoder(nn.Module):

    def __init__(self, input_size, hidden_size, vocab_size):
        super().__init__()

        # TODO: define a single-layer unidirectional LSTM
        self.lstm = ...

        # TODO: define an attention layer
        self.attention = ...

        self.linear_v = nn.Linear(hidden_dim * 3, hidden_dim, bias=True)
        self.linear_v_out = nn.Linear(hidden_dim, vocab_size, bias=True)

    def forward(self, input, prev_h, prev_c, encoder_outputs, pad_mask):

        #TODO: pass the decoder input, previous hidden and cells to LSTM layer
        output, (h_n, c_n) = ...

        #TODO: pass the hidden states (h_n), encoder_outputs and pad_mask to Attention layer
        attn_dist, context = ...

        #TODO: concatenate context vector with hidden state of decoder
        output = ...

        #TOTO:  fed the ouput through two linear layers
        output = ...

        #TODO: apply softmax to output
        vocab_dist = ...

        return vocab_dist, attn_dist, context, h_n, c_n

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size):
        super().__init__()

        # Define a single-layer unidirectional LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Define an attention layer
        self.attention = Attention(hidden_size)

        # Linear layers to produce the vocabulary distribution
        #self.linear_v = nn.Linear(hidden_size * 2, hidden_size, bias=True)  # Concatenated context and hidden state
        self.linear_v = nn.Linear(2 * hidden_size, hidden_size, bias=True)
        self.linear_v_out = nn.Linear(hidden_size, vocab_size, bias=True)

    def forward(self, input, prev_h, prev_c, encoder_outputs, pad_mask):
            # Ensure prev_h and prev_c are 3-dimensional
        if prev_h.dim() == 2:
            prev_h = prev_h.unsqueeze(0)  # Add the num_layers * num_directions dimension
        if prev_c.dim() == 2:
            prev_c = prev_c.unsqueeze(0)  # Add the num_layers * num_directions dimension

        # Pass the decoder input, previous hidden and cells to LSTM layer
        output, (h_n, c_n) = self.lstm(input, (prev_h, prev_c))

        # Pass the hidden states (h_n), encoder_outputs, and pad_mask to the Attention layer
        # We take h_n's last layer output for attention calculation
        attn_dist, context = self.attention(h_n[-1].unsqueeze(1), encoder_outputs, pad_mask)

        if output.size(1) == 1:  # seq_len dimension is 1
            output = output.squeeze(1)

        # Concatenate context vector with hidden state of decoder (h_n)
        # We use output here because it's the output of the LSTM considering the entire sequence
        #combined = torch.cat((output.squeeze(1), context), dim=1)  # Squeezing to remove the sequence length dimension which is 1
        #combined = torch.cat((output.squeeze(1), context), dim=1)  # Now [batch_size, 2 * hidden_size]
        print("Output shape:", output.shape)
        print("Context shape:", context.shape)
        combined = torch.cat((output, context), dim=1)
        print("Combined shape:", combined.shape)

        # Feed the output through two linear layers
        output = self.linear_v(combined)
        output = self.linear_v_out(output)

        # Apply softmax to output to get the vocabulary distribution
        vocab_dist = F.softmax(output, dim=1)

        return vocab_dist, attn_dist, context, h_n, c_n


### Task 5. Pointer-Generator Network (2 points)

In the PGNet class, we will define the embedding layer, encoder, decoder with attention and add a pointer-generator network (3 linear transformations). The pointer-generator network can copy words from the source text via pointing, which aids accurate reproduction of information while retaining the ability to produce novel words through the generator.

**Forward Method:**
  - Iterate over the target sequence length (`TARGET_LEN`).
  - For each time step `t`:
    - Compute the decoder input embedding (`input`) based on the previous decoder output.
    - Use the `AttentionDecoder` to generate the vocabulary distribution (`vocab_dist`), attention distribution (`attn_dist`),  context vector, decoder hidden and cell states.
    - Calculate the generation probability (`p_gen`) based on the context vector, decoder hidden state, and decoder input (Equation 9 in the [ paper](https://arxiv.org/pdf/1704.04368.pdf#page=3)).
    - Compute the final distribution (`final_dist`) by combining the vocabulary distribution and the weighted attention distribution (`w_attn_dist`) based on the generation probability (Equation 9 in the [paper](https://arxiv.org/pdf/1704.04368.pdf#page=3)).
  - Stack the final distributions and attention distributions over time steps.
  - Return the final distributions and attention distributions.

In [44]:
class PGNet(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=PAD_IDX)

        self.encoder = Encoder(embed_size, hidden_size)

        self.decoder = AttentionDecoder(embed_size, hidden_size, vocab_size)

        self.w_context = nn.Linear(hidden_size * 2, 1, bias=False)
        self.w_hidden = nn.Linear(hidden_size, 1, bias=False)
        self.w_input = nn.Linear(hidden_size, 1, bias=True)


    def forward(self, source_tensor, target_tensor, pad_mask, enc_len):

        batch_size=source_tensor.size(0)
        d_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_IDX)

        enc_emb = self.embedding(source_tensor)
        encoder_outputs, (h, c) = self.encoder(enc_emb, enc_len)

        final_dists = []
        attn_dists = []

        #TODO: mention max TARGET_LEN within range function below
        for t in range(...):

            input = self.embedding(d_input)
            vocab_dist, attn_dist, context, h, c = self.decoder(input, h, c, encoder_outputs, pad_mask )

            #TODO: apply the appropiate linear layers to
                 # context vector, decoder hidden states and  decoder input
            context_feature = ...
            decoder_feature  = ...
            input_feature = ...

            #TODO: obtain generation features by adding up all features
            gen_feat =

            #TODO: obtain generation probability by applying sigmoid to generation feature
            p_gen = ...


            #TODO: Calculate new vocab distribution by multiplying to p_gen
            vocab_dist = ...

            #TODO: Calculate weighted attention distribution by
                # multiplying attention distribution to (1-p_gen)
            w_attn_dist = ...

            #TODO:  add vocab_dist to w_attn_dist on the indexes of source_tensor
            final_dist = ...

            final_dists.append(final_dist)
            attn_dists.append(attn_dist)

            #TODO:  If `target_tensor` is provided, use teacher forcing
                  # by feeding the actual target token as the next input;
                  # otherwise, use the predicted token from the current step.
            if target_tensor is not None:
                ...
            else:
                ...

        final_dists = torch.stack(final_dists, dim=-1)
        attn_dists = torch.stack(attn_dists, dim=-1)

        return final_dists, attn_dists

SyntaxError: invalid syntax (<ipython-input-44-f1130554d54f>, line 41)

In [72]:
import torch.nn.functional as F

class PGNet(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=PAD_IDX)
        self.encoder = Encoder(embed_size, hidden_size)
        self.decoder = AttentionDecoder(embed_size, hidden_size, vocab_size)

        # Linear transformations for pointer-generator
        self.w_context = nn.Linear(hidden_size * 2, 1, bias=False)
        self.w_hidden = nn.Linear(hidden_size, 1, bias=False)
        self.w_input = nn.Linear(embed_size, 1, bias=True)

    def forward(self, source_tensor, target_tensor, pad_mask, enc_len, TARGET_LEN):
        batch_size = source_tensor.size(0)
        d_input = torch.empty(batch_size, 1, dtype=torch.long, device=source_tensor.device).fill_(SOS_IDX)

        enc_emb = self.embedding(source_tensor)
        encoder_outputs, (h, c) = self.encoder(enc_emb, enc_len)

        final_dists = []
        attn_dists = []

        for t in range(TARGET_LEN):
            input_emb = self.embedding(d_input)
            vocab_dist, attn_dist, context, h, c = self.decoder(input_emb, h, c, encoder_outputs, pad_mask)

            # Apply linear layers to compute p_gen
            context_feature = self.w_context(context)
            decoder_feature = self.w_hidden(h[-1])  # Using the last layer's h
            input_feature = self.w_input(input_emb.squeeze(1))  # Removing the time dimension

            # Obtain generation feature by summing up all features
            gen_feat = context_feature + decoder_feature + input_feature

            # Obtain generation probability by applying sigmoid to generation feature
            p_gen = torch.sigmoid(gen_feat)

            # Calculate new vocab distribution by multiplying to p_gen
            vocab_dist = p_gen * vocab_dist

            # Calculate weighted attention distribution by multiplying attention distribution to (1 - p_gen)
            w_attn_dist = (1 - p_gen) * attn_dist

            # Add vocab_dist to w_attn_dist on the indexes of source_tensor
            # This is a simplified version assuming the source tokens align with the vocab indices
            for i in range(batch_size):
                final_dist[i, source_tensor[i]] += w_attn_dist[i]

            final_dists.append(final_dist)
            attn_dists.append(attn_dist)

            # Teacher forcing or predicted token as next input
            if target_tensor is not None:
                d_input = target_tensor[:, t].unsqueeze(1)
            else:
                _, top_idx = vocab_dist.max(dim=1)
                d_input = top_idx.unsqueeze(1)  # Use the predicted token as the next input

        final_dists = torch.stack(final_dists, dim=1)  # stacking along time dimension
        attn_dists = torch.stack(attn_dists, dim=1)

        return final_dists, attn_dists


### Task 6. Training [ 1.5 points]

*   Decide on model and training parameters
*   Implement the training loop
   *   Report training and validation loss for each epoch. You can refer to Equations 6 and 7 in the [ paper](https://arxiv.org/pdf/1704.04368.pdf#page=3) regarding loss calculation.
   *   If validation loss is decreased, save the model.
*   Initialize the model and train the model.


In [ ]:
train_vocab

In [73]:
# Model hyperparameters
vocab_size = len(train_vocab)
embed_size = 256
hidden_size = 512
PAD_IDX = 1  # Adjust based on your vocabulary
SOS_IDX = 2  # Adjust based on your vocabulary
TARGET_LEN = 20  # Adjust based on your target sequence lengths

# Training hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [74]:
model = PGNet(vocab_size, embed_size, hidden_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss(ignore_index=PAD_IDX)


In [75]:
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        source_tensor, target_tensor = [item.to(device) for item in batch]

        # Assuming padding index is 0
        pad_mask = (source_tensor == PAD_IDX)
        enc_len = source_tensor.ne(PAD_IDX).sum(dim=1).cpu()  # Moving enc_len to CPU

        output, _ = model(source_tensor, target_tensor, pad_mask, enc_len, TARGET_LEN)

        # Flatten the output and target tensor for loss calculation
        output = output.view(-1, output.shape[-1])
        target_tensor = target_tensor.view(-1)

        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch: {epoch + 1}, Training Loss: {avg_train_loss:.4f}')


{'dialogue': [0, 141, 0, 108, 0, 54, 58, 0, 0, 110, 0, 141, 49, 0, 0, 110, 0, 141, 0, 0, 50, 58, 0, 0, 0, 141, 93, 0, 135, 0, 0, 84, 54, 0, 109, 112, 0, 0, 0, 135, 0, 0, 141, 0, 0, 5, 0, 0, 110, 0, 141, 35, 0, 112, 58, 0, 31, 18, 0, 112, 99, 108, 0, 113, 0, 0, 141, 0, 135, 0, 49, 0, 0, 0, 76, 0, 141, 0, 100, 0, 0, 100, 0, 0, 145, 0, 100, 7, 0, 141, 135, 0, 60, 0, 101, 54, 82, 59, 0, 0, 0, 141, 0, 0, 141, 68, 0, 59, 0, 0, 19, 114, 0, 0, 118, 0, 141, 0, 0, 0, 76, 109, 0, 0, 141, 0, 0, 99, 112, 86, 100, 0, 0, 141, 85, 0, 0, 54, 0, 59, 84, 54, 0, 0, 0, 0, 141, 0, 76, 0, 54, 0, 0, 0, 141, 0, 35, 60, 0, 0, 0, 56, 0, 141, 99, 108, 86, 0, 123, 0, 122, 0, 0, 141, 49, 138, 0, 0, 110, 0, 141, 100, 0, 0, 0, 0, 0, 141, 0, 0, 135, 0, 82, 99, 123, 0, 56, 0, 141, 0, 19, 0, 50, 58, 0, 112, 27, 5, 0, 0, 54, 0, 0, 141, 93, 0, 109, 0, 76, 0, 112, 0, 54, 100, 0, 0, 141, 35, 0, 112, 0], 'summary': [0, 69, 82, 0, 0, 0, 112, 0, 108, 0, 0, 54, 0, 0, 112, 0, 88, 0, 68, 0, 69, 0, 54, 0, 0, 58, 0, 0, 19, 0, 112]}

RuntimeError: batch1 must be a 3D tensor

In [41]:
### viga


num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        # Unpack the batch
        source_tensor, target_tensor = [item.to(device) for item in batch]

        # Generate a pad_mask and calculate enc_len based on source_tensor
        # Assuming padding index is 0
        pad_mask = (source_tensor == PAD_IDX)
        enc_len = source_tensor.ne(PAD_IDX).sum(dim=1)

        output, _ = model(source_tensor, target_tensor, pad_mask, enc_len, TARGET_LEN)

        # Flatten the output and target tensor for loss calculation
        output = output.view(-1, output.shape[-1])
        target_tensor = target_tensor.view(-1)

        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch: {epoch + 1}, Training Loss: {avg_train_loss:.4f}')


{'dialogue': [0, 141, 0, 88, 0, 30, 0, 72, 0, 0, 30, 84, 108, 0, 110, 0, 141, 0, 30, 9, 0, 0, 0, 30, 135, 0, 0, 112, 121, 0, 50, 78, 0, 123, 0, 142, 0, 0, 112, 0, 141, 93, 88, 0, 30, 84, 0, 99, 0, 0, 78, 0, 110, 0, 141, 118, 88, 99, 0, 78, 99, 108, 0, 112, 0, 68, 135, 101, 139, 0, 0, 0, 0, 0, 112, 135, 0, 0, 0, 0, 0, 65, 123, 0, 54, 0, 88, 0, 0, 31, 18, 0, 84, 0, 0, 95, 112, 0, 98, 59, 88, 135, 86, 54, 101, 123, 0, 99, 108, 9, 54, 80, 0, 54, 112, 0, 141, 135, 0, 100, 112, 99, 0, 58, 0, 0, 135, 0, 0, 50, 78, 0, 88, 60, 0, 59, 0, 100, 88, 99, 108, 0, 54, 129, 0, 142, 140, 0, 112, 121, 0, 0, 0, 142, 0, 0, 0, 36, 0, 141, 0, 74, 0, 88, 109, 108, 49, 135, 0, 0, 142, 112, 135, 0, 0, 0, 0, 0, 0, 99, 108, 0, 0, 78, 0, 112, 68, 64, 58, 0, 0, 88, 0, 119, 31, 18, 0, 84, 0, 0, 95, 86, 0, 0, 31, 18, 0, 59, 0, 99, 141, 0, 0, 141, 0, 78, 0, 141, 36, 60, 0, 88, 0, 0, 108, 86, 123, 0, 0, 125, 0, 0, 0, 0, 54, 101, 9, 0, 50, 58, 0, 112, 0, 141, 60, 45, 0, 121, 0, 0, 112, 0, 141, 0, 88, 0, 0, 0, 0, 0, 112]

RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor, but got 1D cuda:0 Long tensor

In [40]:
## viga

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        source_tensor, target_tensor, pad_mask, enc_len = [item.to(device) for item in batch]

        output, _ = model(source_tensor, target_tensor, pad_mask, enc_len, TARGET_LEN)

        # Calculate loss; you might need to adjust dimensions
        output = output.view(-1, vocab_size)  # Flatten output for NLLLoss
        target_tensor = target_tensor.view(-1)  # Flatten target tensor
        loss = criterion(output, target_tensor)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}')

    # Validation loop
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            source_tensor, target_tensor, pad_mask, enc_len = [item.to(device) for item in batch]

            output, _ = model(source_tensor, target_tensor, pad_mask, enc_len, TARGET_LEN)

            # Calculate loss
            output = output.view(-1, vocab_size)  # Flatten output for NLLLoss
            target_tensor = target_tensor.view(-1)  # Flatten target tensor
            loss = criterion(output, target_tensor)

            total_loss += loss.item()

    avg_val_loss = total_loss / len(val_loader)
    print(f'Epoch: {epoch+1}, Validation Loss: {avg_val_loss:.4f}')

    # Save model if validation loss has decreased
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print('Model saved')


{'dialogue': [0, 141, 0, 0, 84, 75, 100, 0, 110, 0, 141, 115, 112, 0, 0, 0, 0, 141, 118, 120, 54, 76, 0, 0, 0, 0, 141, 84, 75, 100, 110, 0, 141, 0, 0, 0, 0, 0, 0, 0, 101, 38, 105, 0, 109, 0, 58, 0, 88, 100, 0, 0, 19, 0, 110, 0, 141, 139, 0, 0, 141, 0, 50, 0, 122, 0, 0, 94, 0, 141, 0, 142, 0, 0, 0, 0, 0, 141, 0, 0, 59, 109, 0, 0, 0, 99, 0, 54, 0, 141, 109, 108, 0, 99, 0, 0, 0, 0, 0, 0, 141, 49, 0, 100, 0, 0, 110, 0, 141, 84, 20, 100, 7, 96, 0, 141, 135, 0, 0, 0, 0, 0, 141, 20, 18, 7, 100, 0, 50, 0, 0, 88, 0, 0, 0, 0, 0, 141, 135, 69, 0, 100, 0, 0, 0, 112, 0, 101, 54, 0, 0, 68, 101, 54, 0, 0, 50, 0, 0, 112, 0, 0, 141, 69, 129, 0, 54, 100, 0, 141, 0, 0, 0, 0], 'summary': [0, 65, 132, 0, 0, 112, 0, 0, 0, 0, 123, 0, 112, 0, 0, 0, 0, 112, 0, 69, 97, 0, 0, 0, 112]}
{'dialogue': [0, 141, 5, 100, 0, 0, 64, 58, 0, 0, 100, 80, 0, 110, 0, 141, 135, 0, 0, 0, 0, 0, 0, 125, 36, 0, 141, 135, 0, 74, 58, 0, 0, 123, 0, 30], 'summary': [0, 108, 0, 0, 123, 0, 68, 0, 108, 0, 0, 0, 0, 112]}
{'dialogue': [0, 

ValueError: not enough values to unpack (expected 4, got 2)

### Task 7. Inference [0.5 points]

*   Create an inference function that gets text as input and returns the summary of the text based on the trained model.
*   Pass at least 10 samples from the test set to the inference function
*   Print the dialogues, human written summary, and generated summary
*   Question: What are the common patterns among the generated summaries?

